\newpage

## Heart Disease Classification Challenge

### Overview

In this assignment, you will analyze the UCI Heart Disease Dataset, which contains medical records used to predict the presence of heart disease in patients. The dataset includes a mix of categorical and numerical variables, some missing values, and class imbalance.  

For the context of data science methods for heart disease prediction, refer to 
- Detrano, R., et., al. (1989). International application of a new probability algorithm for the diagnosis of coronary artery disease. The American journal of cardiology, 64(5), 304-310. DOI:[10.1016/0002-9149(89)90524-9](https://doi.org/10.1016/0002-9149(89)90524-9).

### Dataset Information 

The dataset is available at the UCI Machine Learning Repository:  
🔗 [https://archive.ics.uci.edu/ml/datasets/Heart+Disease](https://archive.ics.uci.edu/ml/datasets/Heart+Disease)  

- Key Features:  
    - The dataset includes 303 observations with 13 features.  
    - Features include age, sex, chest pain type, resting blood pressure, cholesterol levels, fasting blood sugar, electrocardiographic results, and others.  
    - The response variable is `num`, which will be transformed to binary in the analysis.

### Objectives

Analyze the dataset using **two classification algorithms**. Your analysis should include exploratory data analysis, handling of missing values, feature selection, feature engineering, modeling, interpretation, and effective communication. The goal is to draw meaningful and well-supported conclusions from your analysis.

- Classifier requirement: **At least one** of the classifiers must be interpretable to allow for in-depth analysis and inference.

### Assignment Questions

Address the following questions in your submission, providing detailed insights and conclusions based on your analysis.

1. Define and describe a classification problem using the dataset.
2. Apply any chosen data transformations, or explain why no transformations were necessary.
3. Provide a detailed description of the dataset, including variables, summaries, number of observations, data types, and distributions (include at least three statements).
4. Transform the response `num` into a binary outcome: `1` for heart disease and `0` for no heart disease. So combine 1, 2, 3, and 4 into `1` and 0 for `0`. For Questions 4-16, use the transformed binary outcome.
5. Analyze relationships between variables and discuss their implications for feature selection or extraction (include at least two statements).
6. Drop the rows with the missing values. How many osbervations after dropping the missing values. Skip the outlier analysis.
7. Sub-group analysis: Explore potential sub-groups within the data using appropriate data science methods. Identify and visualize these sub-groups without using the labels and categorical variables. Categroical variables already define sub groups so we don't need to include them for this analysis. 
8. Split 30% of the data for testing using a random seed of 1. Use the remaining 70% for training and model selection.
9. Identify the two classifiers you have chosen. Justify your selections based on the classifier requirement for this assignment.
10. Specify two metrics to compare classifier performance. Provide technical details on how each metric is computed.
11. Train two selected classifiers in (9) and identify optimal tuning parameters (if applicable) using the training set.
12. Apply a feature selection or extraction method to one of the classifiers in (9). Train this third classifier on the training set and identify optimal tuning parameters (if applicable) using the training set.
13. Use the selected metrics to evaluate three classifiers in (11) and (12) on the test set.
    - Discuss your findings (at least two statements).
    - Discuss the impact of feature selection or extraction on the performance of the classifier (at least one statement).
14. For the best interpretable model identified in (13), analyze and interpret the most important predictor variables in the context of the classification challenge (at least two statements).
15. **[Bonus]** Sub-group improvement strategy: If sub-groups were identified, propose and implement a method to further improve the performance of **one** classifier. Compare the fourth classifier performance with the results from (13).
16. **Team Contributions:** Document each team member’s specific contributions to the questions above. For group submissions, this should match the GitHub commit history. Individual submissions do not need to address this question.
17. **Link** to the public GitHub repository. This is optional for the individual submissions.



### Notes
- Students can also choose one classifer not covered in the lectures.